In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
!ls /home/m/M.Kreft/Makeathon/data_india/

agriculture_commodity_apmc_msp	crop_recommendation_dataset
crop_production_in_india	data_census_india_centroids


# Show all Datasets

In [3]:
#agriculture_commodity_apmc_msp
cmo_msp_mandi = pd.read_csv("/home/m/M.Kreft/Makeathon/data_india//agriculture_commodity_apmc_msp/CMO_MSP_Mandi.csv")
monthly_data_cmo = pd.read_csv("/home/m/M.Kreft/Makeathon/data_india//agriculture_commodity_apmc_msp/Monthly_data_cmo.csv")

#crop_recommendation_dataset
crop_recommendation = pd.read_csv("/home/m/M.Kreft/Makeathon/data_india//crop_recommendation_dataset/Crop_recommendation.csv")

#crop_production_in_india
crop_production = pd.read_csv("/home/m/M.Kreft/Makeathon/data_india//crop_production_in_india/crop_production.csv")

In [4]:
#print(cmo_msp_mandi["commodity"].unique())
cmo_msp_mandi.head()

,commodity,year,Type,msprice,msp_filter
0,PADDY-UNHUSKED,2012,Kharif Crops,1250.0,1
1,RICE(PADDY-HUS),2012,Kharif Crops,1280.0,1
2,Jowar_Hybrid,2012,Kharif Crops,1500.0,1
3,SORGUM(JAWAR),2012,Kharif Crops,1520.0,1
4,BAJRI,2012,Kharif Crops,1175.0,1


In [5]:
#print(monthly_data_cmo["Commodity"].unique())
monthly_data_cmo.head()

,APMC,Commodity,Year,Month,arrivals_in_qtl,min_price,max_price,modal_price,date,district_name,state_name
0,Ahmednagar,Bajri,2015,April,79,1406,1538,1463,2015-04,Ahmadnagar,Maharashtra
1,Ahmednagar,Bajri,2016,April,106,1788,1925,1875,2016-04,Ahmadnagar,Maharashtra
2,Ahmednagar,Wheat(Husked),2015,April,1253,1572,1890,1731,2015-04,Ahmadnagar,Maharashtra
3,Ahmednagar,Wheat(Husked),2016,April,387,1750,2220,1999,2016-04,Ahmadnagar,Maharashtra
4,Ahmednagar,Sorgum(Jawar),2015,April,3825,1600,2200,1900,2015-04,Ahmadnagar,Maharashtra


In [6]:
print(crop_recommendation["label"].unique())
crop_recommendation.head()

['rice' 'maize' 'chickpea' 'kidneybeans' 'pigeonpeas' 'mothbeans'
 'mungbean' 'blackgram' 'lentil' 'pomegranate' 'banana' 'mango' 'grapes'
 'watermelon' 'muskmelon' 'apple' 'orange' 'papaya' 'coconut' 'cotton'
 'jute' 'coffee']


,N,P,K,temperature,humidity,ph,rainfall,label
0,90,42,43,20.879744,82.002744,6.502985,202.935536,rice
1,85,58,41,21.770462,80.319644,7.038096,226.655537,rice
2,60,55,44,23.004459,82.320763,7.840207,263.964248,rice
3,74,35,40,26.491096,80.158363,6.980401,242.864034,rice
4,78,42,42,20.130175,81.604873,7.628473,262.717340,rice


In [7]:
crop_production.head()

,State_Name,District_Name,Crop_Year,Season,Crop,Area,Production
0,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Arecanut,1254.0,2000.0
1,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Other Kharif pulses,2.0,1.0
2,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Rice,102.0,321.0
3,Andaman and Nicobar Islands,NICOBARS,2000,Whole Year,Banana,176.0,641.0
4,Andaman and Nicobar Islands,NICOBARS,2000,Whole Year,Cashewnut,720.0,165.0


# Dataset4: crop effectivenes in Location:

In [9]:
df = crop_production[['District_Name', 'Crop', 'Area', 'Production']].copy()

f = {
        'District_Name' : 'nunique', 
        'Area' : 'sum',
        'Production' : 'sum'
}

g = df.groupby(['District_Name',"Crop"])
v1 = g.agg(f)
v2 = g.agg(lambda x: x.drop_duplicates('Crop', keep='first').Crop.sum())

df = pd.concat([v1, v2.to_frame('Crop')], 1)

df = df.drop(["Crop","District_Name"], axis=1)
df = df.reset_index(level=['Crop'])
df["efficacy"] = df["Production"]/df["Area"]


In [10]:
df

,Crop,Area,Production,efficacy
District_Name,,,,
24 PARAGANAS NORTH,Arecanut,16591.0,38328.0,2.310168
24 PARAGANAS NORTH,Arhar/Tur,348.0,335.0,0.962644
24 PARAGANAS NORTH,Coconut,37643.0,122664239.8,3258.620190
24 PARAGANAS NORTH,Cotton(lint),4479.0,4786.0,1.068542
24 PARAGANAS NORTH,Dry chillies,46595.0,81182.0,1.742290
...,...,...,...,...
ZUNHEBOTO,Tapioca,1080.0,21440.0,19.851852
ZUNHEBOTO,Tea,395.0,835.0,2.113924
ZUNHEBOTO,Turmeric,50.0,200.0,4.000000


In [11]:
# dist vs best crop
labelled_dist_crop = {}
for dist in df.index.unique():
    argmax = df.loc[dist]["efficacy"].argmax()
    if len(df.loc[dist].shape) > 1:
        crop = df.loc[dist].iloc[int(argmax)]["Crop"]
        labelled_dist_crop[dist] = crop
    else: #if only one entry, suggest that entry
        crop = df.loc[dist]["Crop"]
        labelled_dist_crop[dist] = crop


In [12]:
end_df = pd.DataFrame([labelled_dist_crop])

end_df = end_df.transpose()
end_df = end_df.reset_index()
end_df.rename(columns = {'index':'dist', 0:"crop"}, inplace = True)
end_df.to_csv("districts_vs_crops.csv")
end_df

,dist,crop
0,24 PARAGANAS NORTH,Coconut
1,24 PARAGANAS SOUTH,Coconut
2,ADILABAD,Sugarcane
3,AGAR MALWA,Sugarcane
4,AGRA,Sugarcane
...,...,...
641,YADGIR,Sugarcane
642,YAMUNANAGAR,Sugarcane
643,YANAM,Coconut
644,YAVATMAL,Sugarcane


In [13]:
# dist, best crop, eff
labelled_dist_crop_eff = []
for dist in df.index.unique():
    argmax = df.loc[dist]["efficacy"].argmax()
    if len(df.loc[dist].shape) > 1:
        crop = df.loc[dist].iloc[int(argmax)]["Crop"]
        effi = df.loc[dist].iloc[int(argmax)]["efficacy"]
        labelled_dist_crop_eff.append({"dist":dist, "crop":crop, "eff":effi})
    else: #if only one entry, suggest that entry
        crop = df.loc[dist]["Crop"]
        effi = df.loc[dist]["efficacy"]
        labelled_dist_crop_eff.append({"dist":dist, "crop":crop, "eff":effi})

In [14]:
end_df_eff = pd.DataFrame.from_dict(labelled_dist_crop_eff)
end_df_eff.to_csv("districts_vs_crops_with_efficacy.csv")
no_best_crops = len(set(end_df_eff["crop"]))
end_df_eff.head()

,dist,crop,eff
0,24 PARAGANAS NORTH,Coconut,3258.620190
1,24 PARAGANAS SOUTH,Coconut,2244.891828
2,ADILABAD,Sugarcane,82.385214
3,AGAR MALWA,Sugarcane,37.857143
4,AGRA,Sugarcane,48.458864


In [15]:
end_df_eff.iloc[633]

dist    WEST GODAVARI
crop         Coconut 
eff      14905.128931
Name: 633, dtype: object

In [17]:
set(end_df_eff["crop"])

{'Banana',
 'Barley',
 'Coconut ',
 'Dry chillies',
 'Dry ginger',
 'Maize',
 'Onion',
 'Other Vegetables',
 'Papaya',
 'Pineapple',
 'Potato',
 'Rice',
 'Sugarcane',
 'Sweet potato',
 'Tapioca',
 'Tobacco'}

# read in the price for each crop in mean between 2014-2016

In [18]:
price_df = cmo_msp_mandi
price_df["commodity"] = price_df["commodity"].str.upper()
price_df["msprice"] = price_df["msprice"].fillna(0)

crop_price_dict = []
for crop in list(set(end_df_eff["crop"])):
    if len(price_df[ (price_df["commodity"]==crop.upper()) ]) > 0: #if this series isnt empty
        crop_price_dict.append({crop: np.max(price_df[ (price_df["commodity"]==crop.upper()) ]["msprice"])})
print(crop_price_dict)
no_d1_crops = len(crop_price_dict)

[{'Sugarcane': 230.0}, {'Maize': 1365.0}]


In [19]:
price_df = monthly_data_cmo
price_df["Commodity"] = price_df["Commodity"].str.upper()
crop_price_dict = []
for crop in list(set(end_df_eff["crop"])):
    if len(price_df[ (price_df["Commodity"]==crop.upper()) ]) > 0: #if this series isnt empty
        crop_price_dict.append({crop:np.mean( price_df[ (price_df["Commodity"]==crop.upper() )]["modal_price"] )})
print(crop_price_dict)
no_d2_crops = len(crop_price_dict)

[{'Sweet potato': 1549.0184049079755}, {'Sugarcane': 696.0}, {'Maize': 1355.8165750196385}, {'Other Vegetables': 1306.3866666666668}, {'Onion': 1283.4839743589744}, {'Pineapple': 1578.1150442477876}, {'Potato': 1262.8955431754875}, {'Banana': 886.0364741641338}]


In [20]:
print(no_best_crops, no_d1_crops, no_d2_crops)
# but i know when printing theyre different

16 2 8


# Small calculations to get turnover in Rupee for watermelons and Muskmelons 

In [26]:
df.head()

,Crop,Area,Production,efficacy
District_Name,,,,
24 PARAGANAS NORTH,Arecanut,16591.0,38328.0,2.310168
24 PARAGANAS NORTH,Arhar/Tur,348.0,335.0,0.962644
24 PARAGANAS NORTH,Coconut,37643.0,122664239.8,3258.620190
24 PARAGANAS NORTH,Cotton(lint),4479.0,4786.0,1.068542
24 PARAGANAS NORTH,Dry chillies,46595.0,81182.0,1.742290


In [93]:
monthly_data_cmo["Commodity"].unique()

array(['BAJRI', 'WHEAT(HUSKED)', 'SORGUM(JAWAR)', 'MAIZE', 'GRAM',
       'HORSE GRAM', 'MATKI', 'PIGEON PEA (TUR)', 'BLACK GRAM',
       'CASTOR SEED', 'SOYBEAN', 'JAGGERY', 'LEMON', 'GINGER (FRESH)',
       'POTATO', 'LADIES FINGER', 'FLOWER', 'CARROT', 'CLUSTER BEAN',
       'GHEVDA', 'GHOSALI(BHAJI)', 'MANGO(RAW)', 'CUCUMBER', 'ONION',
       'BITTER GOURD', 'CABBAGE', 'GARLIC', 'MATH (BHAJI)', 'CAPSICUM',
       'TOMATO', 'BRINJAL', 'TAMARIND', 'TAMARIND SEED',
       'CORIANDER (DRY)', 'GREEN CHILLI', 'CHILLIES(RED)', 'MUSTARD',
       'PADDY-UNHUSKED', 'HILDA', 'CHIKOO', 'COTTON',
       'GROUND NUT PODS (DRY)', 'POMEGRANATE', 'PAPAI', 'MELON',
       'BEET ROOT', 'BOTTLE GOURD', 'DHEMSE', 'CORIANDER ', 'CORIANDER  ',
       'SPINACH', 'SHEVGA', 'SMALL GOURD', 'GRAPES', 'KHARBUJ',
       'GREEN GRAM', 'SUNFLOWER', 'SAFFLOWER', 'MANGO', 'WATER MELON',
       'MOSAMBI', 'ORANGE', 'FENUGREEK', 'COWPEA', 'GREEN PEAS (DRY)',
       'SQUASH GOURD', 'MAIZE (CORN.)', 'CHINO', 'CURRY LEA

In [95]:
cmo_msp_mandi["commodity"].unique()

array(['PADDY-UNHUSKED', 'RICE(PADDY-HUS)', 'JOWAR_HYBRID',
       'SORGUM(JAWAR)', 'BAJRI', 'MAIZE', 'RAGI_MALDANDI',
       'PIGEON PEA (TUR)', 'SPILT GERRN GRAM', 'SPLIT BLACK GRAM',
       'COTTON', 'COTTON_LONG STAPLE', 'GR.NUT KERNELS', 'SUNFLOWER',
       'SOYABEAN_BLACK', 'SOYABEAN', 'SESAMUM', 'NIGER-SEED',
       'WHEAT(HUSKED)', 'WHEAT(UNHUSKED)', 'BARLI', 'GRAM_YELLOW',
       'MASUR_YELLOW', 'MUSTARD', 'SAFFLOWER', 'TORIA_YELLOW',
       'COPRA (MILLING)_YELLOW', 'COPRA (BALL)_YELLOW', 'COCONUT',
       'JUTE_YELLOW', 'SUGAR-CANE', 'SUGARCANE'], dtype=object)

In [31]:
df[(df["Crop"]=="Water Melon")] # the databank has some strange entry formats for some crops 
# zero production visible
# Musk Melons are not to be found in this dataset too...
# so now experimental physics inuition:

,Crop,Area,Production,efficacy
District_Name,,,,
ALWAR,Water Melon,116.0,0.0,0.0
BALAGHAT,Water Melon,2.0,0.0,0.0
BARAN,Water Melon,44.0,0.0,0.0
BARWANI,Water Melon,2.0,0.0,0.0
BHARATPUR,Water Melon,4.0,0.0,0.0
...,...,...,...,...
UJJAIN,Water Melon,3.0,0.0,0.0
VELLORE,Water Melon,23.0,0.0,0.0
VIDISHA,Water Melon,10.0,0.0,0.0


In [106]:
watermelon_per_hec = 100*100/1 # one per m^2 is 1e4 per hectar times 
muskmelon_per_hec = 1/0.7 * 1e4 # one muskmelon needs 0.7m^2 to grow, approximately 
area = 100 # in hec

In [107]:
monthly_data_cmo[ (monthly_data_cmo["Commodity"]=="MELON") ] 

,APMC,Commodity,Year,Month,arrivals_in_qtl,min_price,max_price,modal_price,date,district_name,state_name
138,Rahata,MELON,2015,April,3,400,400,400,2015-04,Ahmadnagar,Maharashtra
192,Rahuri,MELON,2016,April,1301,244,1013,631,2016-04,Ahmadnagar,Maharashtra
484,Rahata,MELON,2016,August,37,326,376,326,2016-08,Ahmadnagar,Maharashtra
529,Rahuri,MELON,2016,August,25,100,233,167,2016-08,Ahmadnagar,Maharashtra
1904,Rahata,MELON,2016,July,3,500,500,500,2016-07,Ahmadnagar,Maharashtra
...,...,...,...,...,...,...,...,...,...,...,...
36417,Nashik,MELON,2016,October,20,400,1500,1000,2016-10,Nasik,Maharashtra
36786,Nashik,MELON,2016,September,220,333,833,633,2016-09,Nasik,Maharashtra
48231,Karad,MELON,2016,August,54,2000,2500,2500,2016-08,Satara,Maharashtra
59767,Nashik,MELON,2016,November,250,950,1983,1400,2016-11,Nasik,Maharashtra


In [115]:
price_per_melon = monthly_data_cmo[ (monthly_data_cmo["Commodity"]=="MELON") & (monthly_data_cmo["Year"]==2016) ]["modal_price"].mean()
price_per_melon

1266.7692307692307

In [113]:
turnover_watermelon = price_per_melon * watermelon_per_hec
turnover_muskmelon = price_per_melon * 0.1* muskmelon_per_hec # from indian supermarket sites we found the average 0.1 factor for muskmelon vs watermelon
print(turnover_watermelon)
print(turnover_muskmelon)

12667692.307692308
1809670.3296703298


# Centroid Idea

In [22]:
centroid_dist = pd.read_csv("/home/f/F.Kiessler/makeathon/data_india/data_census_india_centroids/district wise centroids.csv")
centroid_dist.head()

FileNotFoundError: [Errno 2] No such file or directory: '/home/f/F.Kiessler/makeathon/data_india/data_census_india_centroids/district wise centroids.csv'

In [ ]:
print(len(end_df["dist"].unique())- len(centroid_dist["District"].unique()))

In [ ]:
#for disti in centroid_dist["District"]:
print(len(end_df["dist"].iloc[2]), end_df["dist"].iloc[2], type(end_df["dist"].iloc[2]))
print(len(centroid_dist["District"].iloc[2].upper()), centroid_dist["District"].iloc[2].upper(), type(centroid_dist["District"].iloc[2].upper()))
print((centroid_dist["District"].iloc[2].upper() == end_df["dist"].iloc[2]))

In [ ]:
i=0
list_df = list(end_df["dist"])
for disti in centroid_dist["District"]:
    if disti.upper() in list_df:
        continue
    else:
        print(disti.upper())
        i=i+1
print(i)
        

In [ ]:
centroid_dist = pd.read_csv("/home/f/F.Kiessler/makeathon/data_india/data_census_india_centroids/district wise centroids.csv")


# Find a similarity between the district names to make a matching:
Use jellyfish package or python difflib

In [ ]:
!pip install jellyfish

In [ ]:
import jellyfish as j
import difflib as d
s = d.SequenceMatcher(None, "harter_penis","1 2 3r4r 5h")
s.ratio()

In [ ]:
def seqmak(str1,str2):
    return d.SequenceMatcher(None, str1, str2).ratio()
sm=seqmak

In [ ]:

mapping = [[x1,x2,sm(x1,x2)] for x1 in end_df_eff["dist"] for x2 in ["11", "12", "penis"]]

In [ ]:
j.jaro_similarity("penis_hart", "mellon water")

In [ ]:
end_df_eff["dist"]